# Using a Decision Tree to Compress an Image


<img src="output_images_bw/5.png" width="19%" /> <img src="output_images_bw/8.png" width="19%" /> <img src="output_images_bw/14.png" width="19%" /> <img src="output_images_bw/18.png" width="19%" /> <img src="output_images_bw/30.png" width="19%" />

<img src="output_images_hsv/30.png" width="19%" /> <img src="output_images_hsv/18.png" width="19%" /> <img src="output_images_hsv/14.png" width="19%" /> <img src="output_images_hsv/8.png" width="19%" /> <img src="output_images_hsv/5.png" width="19%" />

<img src="images/animated_comparison_rgb_hsv.gif" width="80%" >

In [ ]:
# | hide
from nbdev.showdoc import *

# todo: 
# skip_showdoc: true
# skip_exec: true


# Import Libraries

In [ ]:
%config InlineBackend.figure_format = 'retina'
import pathlib
import warnings

import cv2
import numpy as np
from IPython.display import Markdown, display
from joblib import Memory, dump, load
from PIL import Image, ImageDraw, ImageFont, ImageSequence
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

warnings.simplefilter(action="ignore", category=FutureWarning)

# Define Constants

In [ ]:
# For the final gif
font = ImageFont.truetype("/System/Library/Fonts/HelveticaNeue.ttc", 250)
left_pad = 80
upper_pad = 40
lower_pad = 100

# Load Image and Prepare Data

In [ ]:
def process_image_to_bw(data):
    """
    Convert an image to black and white and create a coordinate array and a one-dimensional representation of the image.

    Parameters:
    data (numpy.ndarray): The input image data.

    Returns:
    numpy.ndarray: The coordinate array for the image.
    numpy.ndarray: The one-dimensional representation of the image.
    """
    # Convert the image to grayscale
    data_bw = cv2.cvtColor(data, cv2.COLOR_RGB2GRAY)

    # Get the dimensions of the image
    rows_bw, cols_bw = data_bw.shape

    # Create a coordinate array for the image
    X_bw = np.array([(row, col) for row in range(rows_bw) for col in range(cols_bw)])

    # Reshape the data array into a column vector
    y_bw = data_bw.reshape(-1, 1)

    return X_bw, y_bw


def process_image_to_rgb(data):
    """
    Create a coordinate array and reshape the data array into a column vector.

    Parameters:
    data (numpy.ndarray): The input image data.

    Returns:
    numpy.ndarray: The coordinate array for the image.
    numpy.ndarray: The reshaped data array into a column vector.
    """
    # Create coordinate arrays using mgrid
    row_coords, col_coords, band_coords = np.mgrid[
        : data.shape[0], : data.shape[1], : data.shape[2]
    ]

    # Reshape the coordinate arrays into a single array
    X_rgb = np.column_stack(
        (row_coords.ravel(), col_coords.ravel(), band_coords.ravel())
    )

    # Reshape the data array into a column vector
    y_rgb = data.reshape(-1, 1)

    return X_rgb, y_rgb


def process_image_to_hsv(data):
    """
    Convert an image to HSV and create a coordinate array and a two-dimensional representation of hue and saturation.

    Parameters:
    data (numpy.ndarray): The input image data.

    Returns:
    numpy.ndarray: The coordinate array for the image.
    numpy.ndarray: The two-dimensional representation of hue and saturation.
    """
    # Convert the image to HSV
    data_hsv = cv2.cvtColor(data, cv2.COLOR_RGB2HSV)

    # Get the dimensions of the image
    rows_hsv, cols_hsv, bands_hsv = data_hsv.shape

    # Create a coordinate array for the image
    X_hsv = np.array([(row, col) for row in range(rows_hsv) for col in range(cols_hsv)])

    # Extract hue, saturation, and value
    hue, sat, val = cv2.split(data_hsv)
    hue = hue / 179.0 * np.pi * 2.0  # Convert hue to radians

    # Create two-dimensional representation of hue and saturation
    hue_x = np.cos(hue).reshape(-1, 1)
    hue_y = np.sin(hue).reshape(-1, 1)
    sat = sat.reshape(-1, 1)
    val = val.reshape(-1, 1)

    # Stack hue_x, hue_y, sat, and val to create y_hsv
    y_hsv = np.hstack((hue_x, hue_y, sat, val))

    return X_hsv, y_hsv

## Process the image to create the coordinate array and the target array

In [ ]:
data = np.asarray(Image.open("images/vangogh-original.jpg"))
X_bw, y_bw = process_image_to_bw(data)
X_rgb, y_rgb = process_image_to_rgb(data)
X_hsv, y_hsv = process_image_to_hsv(data)

## Convert to black and white

In [ ]:
def convert_to_bw_image(data):
    """
    Convert an image to black and white.

    Parameters:
    data (numpy.ndarray): The input image data.

    Returns:
    PIL.Image.Image: The black and white image.
    """
    # Convert the image to grayscale
    data_bw = cv2.cvtColor(data, cv2.COLOR_RGB2GRAY)

    # Convert the grayscale array to an image
    bw_image = Image.fromarray(data_bw.astype(np.uint8))

    return bw_image


bw_image = convert_to_bw_image(data)
bw_image.save("images/vangogh-bw.jpg")

# Build the models

In [ ]:
memory = Memory("cache_directory", verbose=0)
max_depths = range(1, 31)

In [ ]:
@memory.cache
def create_regressor_and_bw_image(X_bw, y_bw, data, max_depth):
    """
    Create a Decision Tree Regressor, fit it to the black and white data, make predictions, and convert the predictions to an image.

    Parameters:
    X_bw (numpy.ndarray): The input features for the regressor.
    y_bw (numpy.ndarray): The target output for the regressor.
    data (numpy.ndarray): The original image data used to reshape the predicted array.
    max_depth (int): The maximum depth of the tree.

    Returns:
    DecisionTreeRegressor: The fitted regressor.
    PIL.Image.Image: The image created from the predictions.
    """
    # Create the regressor
    regressor = DecisionTreeRegressor(max_depth=max_depth)

    # Fit the regressor to the black and white data
    regressor.fit(X_bw, y_bw)

    # Make predictions
    y_pred = regressor.predict(X_bw)

    # Reshape the predictions to the original image shape
    y_pred_reshaped = y_pred.reshape(data.shape[0], data.shape[1])

    # Stack the reshaped predictions to create a 3-channel image
    bw_image_rgb = np.stack((y_pred_reshaped,) * 3, axis=-1)

    # Convert the 3-channel image to a PIL Image
    image = Image.fromarray(bw_image_rgb.astype(np.uint8))

    return regressor, image


@memory.cache
def create_regressor_and_rgb_image(X, y, data, max_depth):
    """
    Create a Decision Tree Regressor, fit it to the data, make predictions, and convert the predictions to an image.

    Parameters:
    X (numpy.ndarray): The input features for the regressor.
    y (numpy.ndarray): The target output for the regressor.
    data (numpy.ndarray): The original image data used to reshape the predicted array.
    max_depth (int): The maximum depth of the tree.

    Returns:
    DecisionTreeRegressor: The fitted regressor.
    PIL.Image.Image: The image created from the predictions.
    """
    # Create the regressor
    regressor = DecisionTreeRegressor(max_depth=max_depth)

    # Fit the regressor to the data
    regressor.fit(X, y)

    # Make predictions
    y_pred = regressor.predict(X)

    # Reshape the predictions to the original image shape and convert to an image
    image = Image.fromarray(y_pred.reshape(data.shape).astype(np.uint8))

    return regressor, image


@memory.cache
def create_regressor_and_hsv_image(X_hsv, y_hsv, data, max_depth):
    """
    Create a Decision Tree Regressor, fit it to the HSV data, make predictions, convert the predictions to HSV format, and convert the predictions to an image.

    Parameters:
    X_hsv (numpy.ndarray): The input features for the regressor.
    y_hsv (numpy.ndarray): The target output for the regressor.
    data (numpy.ndarray): The original image data used to reshape the predicted array.
    max_depth (int): The maximum depth of the tree.

    Returns:
    DecisionTreeRegressor: The fitted regressor.
    PIL.Image.Image: The image created from the predictions.
    """
    # Create the regressor
    regressor = DecisionTreeRegressor(max_depth=max_depth)

    # Fit the regressor to the HSV data
    regressor.fit(X_hsv, y_hsv)

    # Make predictions
    y_pred = regressor.predict(X_hsv).astype(np.float32)

    # Convert the predictions to HSV format
    y_pred_hue = (
        np.arctan2(y_pred[:, 1], y_pred[:, 0]) % (2 * np.pi) / (2 * np.pi) * 179.0
    ).astype(np.uint8)  # Convert hue to degrees
    y_pred_sat = y_pred[:, 2].astype(np.uint8)
    y_pred_val = y_pred[:, 3].astype(np.uint8)
    y_pred = cv2.merge((y_pred_hue, y_pred_sat, y_pred_val))

    # Reshape the predictions to the original HSV image shape, convert to RGB format, and convert to an image
    image = Image.fromarray(
        cv2.cvtColor(y_pred.reshape(data.shape), cv2.COLOR_HSV2RGB).astype(np.uint8)
    )

    return regressor, image

In [ ]:
# | notest
bw_model_results = {}
rgb_model_results = {}
hsv_model_results = {}

for max_depth in max_depths:
    bw_model_results[max_depth] = {}
    (
        bw_model_results[max_depth]["regressor"],
        bw_model_results[max_depth]["image"],
    ) = create_regressor_and_bw_image(X_bw, y_bw, data, max_depth)

    rgb_model_results[max_depth] = {}
    (
        rgb_model_results[max_depth]["regressor"],
        rgb_model_results[max_depth]["image"],
    ) = create_regressor_and_rgb_image(X_rgb, y_rgb, data, max_depth)

    hsv_model_results[max_depth] = {}
    (
        hsv_model_results[max_depth]["regressor"],
        hsv_model_results[max_depth]["image"],
    ) = create_regressor_and_hsv_image(X_hsv, y_hsv, data, max_depth)

In [ ]:
# create output_images_bw directory if it doesn't exist
pathlib.Path("output_images_bw").mkdir(parents=True, exist_ok=True)
# create output_images_rgb directory if it doesn't exist
pathlib.Path("output_images_rgb").mkdir(parents=True, exist_ok=True)
# create output_images_hsv directory if it doesn't exist
pathlib.Path("output_images_hsv").mkdir(parents=True, exist_ok=True)


In [ ]:
# | notest
for max_depth in max_depths:
    hsv_model_results[max_depth]["image"].save(f"output_images_hsv/{max_depth}.png")
    rgb_model_results[max_depth]["image"].save(f"output_images_rgb/{max_depth}.png")
    bw_model_results[max_depth]["image"].save(f"output_images_bw/{max_depth}.png")

In [ ]:
# | notest
# List to store the concatenated images
concat_images = []

for max_depth in max_depths:
    img_rgb = rgb_model_results[max_depth]["image"]
    img_hsv = hsv_model_results[max_depth]["image"]
    total_width = img_rgb.width + img_hsv.width
    concat_img = Image.new("RGB", (total_width, img_rgb.height))
    concat_img.paste(img_rgb, (0, 0))
    concat_img.paste(img_hsv, (img_rgb.width, 0))

    # create draw object
    draw = ImageDraw.Draw(concat_img)

    # Define text and font
    text_depth = f"max_depth = {max_depth:02d}"
    text_rgb_label = "RGB"
    text_hsv_label = "HSV"

    # Get bounding box for text to calculate width and height
    bbox = draw.textbbox((0, 0), text_depth, font=font)
    width, height = bbox[2] - bbox[0], bbox[3] - bbox[1]

    # Set position for the depth text to be centered
    position_depth = ((total_width - width) // 2, img_rgb.height - height - lower_pad)

    # Set positions for RGB and HSV labels
    position_rgb = (left_pad, upper_pad)
    position_hsv = (img_rgb.width + left_pad, upper_pad)

    # Draw the texts on the image
    draw.text(position_rgb, text_rgb_label, fill="white", font=font)
    draw.text(position_hsv, text_hsv_label, fill="white", font=font)
    draw.text(position_depth, text_depth, fill="white", font=font)

    # Add concatenated image to the list
    concat_images.append(concat_img)

# Create animated gif
concat_images[0].save(
    "images/animated_comparison_rgb_hsv.gif",
    save_all=True,
    append_images=concat_images[1:],
    loop=0,
    duration=300,
)

## Cleanup

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()